In [1]:
import pickle as pkl
import numpy as np
from tqdm import tqdm
import os
import json

In [11]:
in_dir = "data/pkl_fine"
in_file = "cv.pkl"
out_dir = "data/json/"
out_file = "data_fine.json"

In [12]:
in_path = f'{in_dir}/{in_file}'
out_path = f'{out_dir}/{out_file}'

In [22]:
with open(in_path, "rb") as f_in, open(out_path, "w") as f_out:
    data = pkl.load(f_in)
    json_format = data.json_format
    print((data.wrapper.render_dpi))
    
    buf_idx = 0
    print(json_format[buf_idx])
    print(len(lines))
    linebuf = lines[json_format[buf_idx]['page']][json_format[buf_idx]['idx_in_page']]
    fontname = linebuf.fontname.lower()
    boldbuf = 1 if "bold" in fontname else 0
    italbuf = 1 if "italic" in fontname else 0

100
{'text': 'Jeff Erickson', 'page': 0, 'idx_in_page': 0, 'original_dimensions': {'x0': 357.00972222222225, 'y0': 77.75353458333335, 'width': 135.97966055555563, 'height': 22.954027777777714}, 'x': 42.001143790849675, 'y': 7.068503143939394, 'width': 15.997607124183014, 'height': 2.0867297979797925}
0


IndexError: list index out of range

In [32]:
from pdf_wrapper import PDFWrapper
def get_record(in_path):
    with open(in_path, "rb") as f_in, open(out_path, "w") as f_out:
        data = pkl.load(f_in)
        json_format = data.json_format
        pdf_path = in_path.replace("pkl_fine", "pdf")
        pdf_path = pdf_path.replace(".pkl", ".pdf")
        wrapper = PDFWrapper(fname = pdf_path)
        lines = wrapper.lines
        for record in tqdm(data.record):
            # print(record)
            buf_idx = record['from']
            stk_idx = record['to']
            type_str = record['type']
            buf_string = None
            stk_string = None
            lbuf = None
            rbuf = None 
            lstk = None 
            rstk = None 
            hstk = None
            boldbuf = None
            italbuf = None
            boldstk = None
            italstk = None
            hbuf = None
            if(buf_idx == -1):
                buf_string = "$ROOT"
                lbuf = 0
                rbuf = 100
                hbuf = 30
                boldbuf = 0
                italbuf = 0
            else:
                buf_string = "$ROOT" if buf_idx == -1 else json_format[buf_idx]['text']
                lbuf = json_format[buf_idx]['x']
                rbuf = json_format[buf_idx]['x'] + json_format[buf_idx]['width']
                hbuf = int(json_format[buf_idx]['height'])
                try:
                    linebuf = lines[json_format[buf_idx]['page']][json_format[buf_idx]['idx_in_page']]
                except:
                    print(f"Tried to get line #{json_format[buf_idx]['idx_in_page']} of page {json_format[buf_idx]['page']}; document has {len(wrapper.elements)}/{len(wrapper.lines)} pages, and that page has {len(lines[json_format[buf_idx]['idx_in_page']])} lines")
                    raise KeyError
                fontname = linebuf._objs[0].fontname.lower()
                boldbuf = 1 if "bold" in fontname else 0
                italbuf = 1 if "italic" in fontname else 0

            if(stk_idx == -1):
                stk_string = "$ROOT"
                lstk = 0
                rstk = 100
                hstk = 30
                boldbuf = 0
                italbuf = 0
            else:
                stk_string = "$ROOT" if stk_idx == -1 else json_format[stk_idx]['text']
                lstk = json_format[stk_idx]['x']
                rstk = json_format[stk_idx]['x'] + json_format[stk_idx]['width']
                hstk = int(json_format[stk_idx]['height'])
                linebuf = lines[json_format[stk_idx]['page']][json_format[stk_idx]['idx_in_page']]
                fontname = linebuf._objs[0].fontname.lower()
                boldbuf = 1 if "bold" in fontname else 0
                italbuf = 1 if "italic" in fontname else 0
            yield {
                'buf_str': buf_string,
                'lbuf': lbuf,
                'rbuf': rbuf,
                'hbuf': hbuf,
                'boldbuf': boldbuf,
                'italbuf': italbuf,
                'stk_str': stk_string,
                'lstk': lstk,
                'rstk': rstk,
                'boldstk': boldstk,
                'italstk': italstk,
                'type': type_str,
                'hstk': hstk
            }
            # print(f'{buf_string},{stk_string},{type_str}')


In [5]:
# # fix a bug where the discard function had the from and to indices both point to themselves 

# def fix_discard(in_path):
    
#     data = None
#     print(in_path)
#     with open(in_path, "rb") as f_in:
#         data = pkl.load(f_in)

#     stack = [-1] 

#     for entry in data.record:
#         print("ORIGINAL", entry)
#         if(entry['type'] == 'pop'):
#             stack.pop()
#         if(entry['type'] == 'merge'):
#             stack.pop()
#             stack.append(entry['from'])
#         if(entry['type'] == 'subordinate'):
#             stack.append(entry['from'])
#         if(entry['type'] == 'discard'):
#             entry['to'] = stack[-1]
#         entry['to'] = 

#     with open(in_path, "wb") as f_out:
#         pkl.dump(data, f_out)
#         # above line is pretty dangerous, should wait until everything is ready


# def lengthen_sentences(in_path, out_path):
#     data = None
#     print(in_path)
#     with open(in_path, "rb") as f_in:
#         data = pkl.load(f_in)

#     stack = [-1] 
#     current_text = ""
#     for entry in data.record:
#         print("ORIGINAL", entry)
#         if(entry['type'] == 'pop'):
#             stack.pop()
#         if(entry['type'] == 'merge'):
#             stack.pop()
#             stack.append(entry['from'])
#         if(entry['type'] == 'subordinate'):
#             stack.append(entry['from'])
#         if(entry['type'] == 'discard'):
#             entry['to'] = stack[-1]
#         entry['to'] = 

#     with open(out_path, "wb") as f_out:
#         pkl.dump(data, f_out)
#         # above line is pretty dangerous, should wait until everything is ready

SyntaxError: invalid syntax (1777237566.py, line 23)

In [33]:
def pkl_to_json(in_dir = "data/pkl_fine", out_dir = "data/json/", out_name = "data_fine.json"):
    to_write = {
        'buf_str': [],
        'lbuf': [],
        'rbuf': [],
        'hbuf': [],
        'boldbuf': [],
        'italbuf': [],
        'stk_str': [],
        'lstk': [],
        'rstk': [],
        'hstk': [],
        'boldstk': [],
        'italstk': [],
        'type': []
    }

    for file in os.listdir(in_dir):
        full_path = f'{in_dir}/{file}'
        print(full_path)
        if(os.path.isfile(full_path)):
            for entry in get_record(full_path):
                for key, val in entry.items():
                    to_write[key].append(val)
                    # print(key, val)
    # print(to_write)
    print(f'Found {len(to_write["buf_str"])} elements')
    out_path = f'{out_dir}/{out_name}'  

    with open(out_path, "w") as f_out:
        json.dump(to_write, f_out)

In [34]:
pkl_to_json()


data/pkl_fine/Yoon-current-cv-web.pkl
Using laparams =  <LAParams: char_margin=2.0, line_margin=0.5, word_margin=0.1 all_texts=False>


36it [00:01, 22.85it/s]
100%|██████████| 1996/1996 [00:00<00:00, 203357.72it/s]


data/pkl_fine/cv (1).pkl
Using laparams =  <LAParams: char_margin=2.0, line_margin=0.5, word_margin=0.1 all_texts=False>


17it [00:03,  5.12it/s]
  4%|▍         | 47/1163 [00:00<00:00, 138241.44it/s]

Tried to get line #32 of page 0; document has 142 pages, and that page has 33 lines


KeyError: 